# 基础知识
huggingface是一个平台，本身提供了一个transformers的底层机器学习库，同时还提供了很多开源的models和datasets。我们主要用到了里面的预训练模型，开箱即用，可以直接用到我们的工作中去。

## 模型model
huggingface上面有很多模型，包括预训练的模型，主要是解决NLP和图像以及音频类的任务。
这些任务叫做[task](https://huggingface.co/tasks)。

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 
1
2

2

# 任务场景


# 常见问题